# Initialization

Test notebook for the C-MAPPS benchmark. Approach using MLP with Time window. 

First we import the necessary packages and create the global variables.

In [ ]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

# Single MLP

Use the MLP with 1 hidden layer and 20 Neurons as per reference "A Time Window Neural Network Based Framework for Remaining Useful Life Estimation"

In [2]:
def RULmodel_TW(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Reshape the data according to the paper

Use different time window sizes for the comparisson. "A Time Window Neural Network Based Framework for Remaining Useful Life Estimation"

In [ ]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

#Selected from the first section of the paper
windowSize = 1
stride = 1
constRUL = 105

dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_1(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)